<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Rag_Ollama3_Chromadb2_23_10_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install ollama
#!pip install chromadb
#!pip install feedparser
#!pip install streamlit

In [ ]:
import ollama
import chromadb
import feedparser

rss_feed_link = "https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml"
collection_name = "nyt_tech_feed"

feed = feedparser.parse(rss_feed_link)
entries = feed.entries
# print(entries)

documents = []
metadatas = []
ids = []


for entry in entries:
    title = entry.title
    link = entry.link
    content = entry.summary
    tags = ", ".join(t["term"] for t in entry.tags)

    documents.append(f"# {title}\n{content}")
    metadatas.append({"title": title, "link": link, "tags": tags})
    ids.append(link)


# print(documents[0])
# print(metadatas[0])
# print(ids[0])

client = chromadb.Client()

collection = client.get_or_create_collection(collection_name)

collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

prompt = "list top 3 most intersting AI related news with summaries and link"
query_result = collection.query(query_texts=[prompt])
context = query_result["documents"][0]

stream = ollama.chat(
model = "llama3:instruct",
messages= [
    {"role": "system",
    "content" : f"Answer the questions based on the news feed given here only. If you donot know the answer, say I donot know. The news feed content is here: \n\n------------------{context}-------------------\n\n "
    },
    {
        "role": "user",
        "content": prompt
    }
],
    stream= True

)

for chunk in stream:
    print(chunk["message"]["content"], end = "")